In [ ]:
# chat gpt spectral mixture analysis
import ee
from ee import batch

# Initialize Earth Engine
ee.Initialize()

# Define a region of interest (ROI) for the analysis
roi = ee.Geometry.Rectangle([-122.45, 37.74, -122.4, 37.75])  # Example ROI in San Francisco Bay Area

# Function to perform Spectral Mixture Analysis (SMA) on an image
def perform_sma(image):
    # Select bands of interest (typically, bands 2-7 for Landsat 8/9)
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

    # Endmembers for SMA (example endmembers, can be adjusted)
    water_endmember = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
    vegetation_endmember = [0.3, 0.4, 0.5, 0.2, 0.1, 0.0]
    soil_endmember = [0.6, 0.5, 0.4, 0.1, 0.0, 0.0]

    # Create an SMA endmember array
    endmembers = ee.Array([water_endmember, vegetation_endmember, soil_endmember])

    # Perform SMA
    sma = image.unmix(endmembers).rename(['water', 'vegetation', 'soil'])

    return sma

# Load Landsat 9 collection
landsat9 = ee.ImageCollection('LANDSAT/LC09/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .median()  # Take median to reduce noise

# Perform SMA on the Landsat 9 image
sma_output = perform_sma(landsat9)

# Define output directory in Google Drive
output_dir = 'Spectral_Mixture_Analysis_Output'

# Export the SMA result to Google Drive
task = batch.Export.image.toDrive(
    image=sma_output,
    description='SMA_Output_Landsat9',
    folder=output_dir,
    scale=30,  # Adjust scale as needed
    region=roi
)

# Start the export task
task.start()

# Print task status
print('Exporting SMA result to Google Drive...')